In [1]:
!python3 -m pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.27.1
    Uninstalling openai-0.27.1:
      Successfully uninstalled openai-0.27.1

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
import os
import openai
import constants
openai.api_key = constants.GPT_API_KEY

In [3]:
dataset_folder = './dataset'

In [4]:
def get_responses(song_name, genre):
    prompt = f'The string on the line below is the YouTube video title of a piano cover of a(n) {genre} song.\
                 Given the string, write a rich musical description of the original song, not the cover.\
                 This description should be roughly 200 words.\
                 \n{song_name}'

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "user", "content": prompt}
            ]
    )
    caption = response['choices'][0]['message']['content'].lstrip('\n')

    return caption

In [7]:
def caption_all_genre_folders(overwrite=False):
    genre_list = os.listdir(dataset_folder)
    for genre in genre_list:
        if genre.startswith('.') or not os.path.isdir(f'{dataset_folder}/{genre}'):
            continue
        
        caption_genre_folder(genre, overwrite)


def caption_genre_folder(genre, overwrite=False):
    '''
    Takes in the name of a genre folder containing .mp3 files and generates rich text captions
    for each song in the genre folder in .txt format

            Parameters:
                    genre (str): the genre whose songs to caption
                    overwrite (bool): specify whether captioning should overwrite existing 
                                      .txt files with the same file name
    '''
    genre_path = f'{dataset_folder}/{genre}'

    song_list = os.listdir(genre_path)

    for song_name in song_list:
        if song_name.startswith('.'):
            continue
        caption_filename = song_name + '_caption.txt'
        if not overwrite and os.path.isfile(f'{genre_path}/{song_name}/{caption_filename}'):
            continue

        song_path = f'{genre_path}/{song_name}'
        if not os.path.exists(song_path):
            os.mkdir(song_path)

        caption = get_responses(song_name, genre)
        assert(caption != None and len(caption) != 0)

        with open(f'{song_path}/{caption_filename}', 'w') as f:
            f.write(caption)

In [8]:
caption_all_genre_folders()